In [3]:
import json
import numpy as np
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import interp1d
import pandas as pd
import os
import matplotlib.pyplot as plt

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return 360 - angle if angle > 180.0 else angle

def interpolate_signal(signal, valid_indices, all_indices):
    if len(valid_indices) < 2:
        return signal
    interp_func = interp1d(valid_indices, signal[valid_indices], kind='linear', fill_value='extrapolate')
    interpolated = np.copy(signal)
    interpolated[np.isnan(signal)] = interp_func(all_indices[np.isnan(signal)])
    return interpolated

def get_landmark_by_name(landmarks, name):
    for lm in landmarks:
        if lm['name'] == name:
            return lm
    return None

def count_reps_from_json(json_path, confidence_threshold=0.6, frame_rate=30, visualize=False):
    with open(json_path, 'r') as f:
        data = json.load(f)

    left_angles, right_angles, combined_angles = [], [], []
    left_confidences, right_confidences = [], []
    frame_indices = []

    for frame_index, frame in enumerate(data):
        landmarks = frame.get("landmarks", [])
        # Get required keypoints by name
        ls = get_landmark_by_name(landmarks, "left_shoulder")
        le = get_landmark_by_name(landmarks, "left_elbow")
        lw = get_landmark_by_name(landmarks, "left_wrist")

        rs = get_landmark_by_name(landmarks, "right_shoulder")
        re = get_landmark_by_name(landmarks, "right_elbow")
        rw = get_landmark_by_name(landmarks, "right_wrist")

        # Check confidence and calculate left elbow angle
        if ls and le and lw and min(ls['conf'], le['conf'], lw['conf']) > confidence_threshold:
            left_angle = calculate_angle((ls["x"], ls["y"]),
                                         (le["x"], le["y"]),
                                         (lw["x"], lw["y"]))
            left_conf = min(ls['conf'], le['conf'], lw['conf'])
        else:
            left_angle = np.nan
            left_conf = 0

        # Right elbow angle
        if rs and re and rw and min(rs['conf'], re['conf'], rw['conf']) > confidence_threshold:
            right_angle = calculate_angle((rs["x"], rs["y"]),
                                          (re["x"], re["y"]),
                                          (rw["x"], rw["y"]))
            right_conf = min(rs['conf'], re['conf'], rw['conf'])
        else:
            right_angle = np.nan
            right_conf = 0

        left_angles.append(left_angle)
        right_angles.append(right_angle)
        left_confidences.append(left_conf)
        right_confidences.append(right_conf)

        # Combined average angle
        if not np.isnan(left_angle) and not np.isnan(right_angle):
            combined_angles.append((left_angle + right_angle) / 2)
        elif not np.isnan(left_angle):
            combined_angles.append(left_angle)
        elif not np.isnan(right_angle):
            combined_angles.append(right_angle)
        else:
            combined_angles.append(np.nan)

        frame_indices.append(frame_index)

    if not combined_angles:
        print(f"No valid keypoints in {os.path.basename(json_path)}")
        return 0, []

    left_angles, right_angles, combined_angles = map(np.array, [left_angles, right_angles, combined_angles])
    frame_indices = np.array(frame_indices)

    # Interpolate missing data
    for signal in [left_angles, right_angles, combined_angles]:
        valid_indices = np.where(~np.isnan(signal))[0]
        if len(valid_indices) >= 2:
            signal[:] = interpolate_signal(signal, valid_indices, frame_indices)

    # Smooth angles
    left_angles = gaussian_filter1d(left_angles, sigma=4)
    right_angles = gaussian_filter1d(right_angles, sigma=4)
    combined_angles = gaussian_filter1d(combined_angles, sigma=4)

    # Detect troughs (elbow flexion peaks)
    distance = int(frame_rate * 0.5)  # at least 0.5 sec apart
    peaks, _ = find_peaks(-combined_angles, height=-100, distance=distance, prominence=10)
    rep_count = len(peaks)

    avg_left_conf = np.mean(left_confidences) if left_confidences else 0
    avg_right_conf = np.mean(right_confidences) if right_confidences else 0
    print(f"{os.path.basename(json_path)}: Reps={rep_count}, Peaks={list(peaks)}, "
          f"Left Conf={avg_left_conf:.2f}, Right Conf={avg_right_conf:.2f}")

    if visualize:
        plt.figure(figsize=(12, 8))
        plt.plot(left_angles, label="Left Elbow Angle", alpha=0.5)
        plt.plot(right_angles, label="Right Elbow Angle", alpha=0.5)
        plt.plot(combined_angles, label="Combined Elbow Angle", linewidth=2)
        plt.plot(peaks, combined_angles[peaks], "x", label="Troughs", color="red")
        plt.title(f"Elbow Angles for {os.path.basename(json_path)}")
        plt.xlabel("Frame")
        plt.ylabel("Angle (degrees)")
        plt.legend()
        plt.show()

    return rep_count, peaks.tolist()

def process_json_folder(json_dir, visualize=False):
    results = []
    json_files = [f for f in os.listdir(json_dir) if f.endswith('.json')]

    for json_file in json_files:
        json_path = os.path.join(json_dir, json_file)
        try:
            rep_count, peaks = count_reps_from_json(json_path, visualize=visualize)
            peak_str = ",".join(map(str, peaks))
            results.append({'JSON_File': json_file, 'Rep_Count': rep_count, 'Peak_Frames': peak_str})
        except Exception as e:
            print(f"Error processing {json_file}: {e}")
            results.append({'JSON_File': json_file, 'Rep_Count': 0, 'Peak_Frames': ''})

    df = pd.DataFrame(results)
    output_csv = os.path.join(json_dir, 'rep_counts_from_json.csv')
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")
    return df

if __name__ == "__main__":
    json_folder = r"C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Test_Bhaskar_Sir\barbell_bicep_curls\Json"
    df = process_json_folder(json_folder, visualize=False)
    print(df)


barbell_biceps_curl_segment_10_start_18m21s869ms_pose.json: Reps=7, Peaks=[24, 167, 379, 449, 528, 603, 698], Left Conf=0.68, Right Conf=0.95
barbell_biceps_curl_segment_1_start_00m27s730ms_pose.json: Reps=1, Peaks=[530], Left Conf=0.77, Right Conf=0.97
Results saved to C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Test_Bhaskar_Sir\barbell_bicep_curls\Json\rep_counts_from_json.csv
                                           JSON_File  Rep_Count  \
0  barbell_biceps_curl_segment_10_start_18m21s869...          7   
1  barbell_biceps_curl_segment_1_start_00m27s730m...          1   

                  Peak_Frames  
0  24,167,379,449,528,603,698  
1                         530  
